In [1]:
import json
import os
from os.path import join as pjoin
import cv2
import sys

In [2]:
def draw_label(img, bound, color, text=None, put_text=True):
    cv2.rectangle(img, (bound[0], bound[1]), (bound[2], bound[3]), color, 3)
    if put_text:
        # put text with rectangle 
        (w,h),_ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1.6, 3)
        cv2.rectangle(img, (bound[0], bound[1] - 40), (bound[0] + w, bound[1] - 40 + h), color, -1)
        cv2.putText(img, text, (bound[0], bound[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 3) 


def visualize_list(img, data, show=True):
    board = img.copy()
    is_drawn = False
    
    if data['vertical_groups'] is not None:
        for g in data['vertical_groups']:
            draw_label(board, data['bounds'][g[0]], (0,0,166), 'vertical_groups')
            for compo_id in g[1:]:
                b = data['bounds'][compo_id]
                draw_label(board, b, (0,0,166), put_text=False)
                is_drawn = True

    if data['horizontal_groups'] is not None:
        for g in data['horizontal_groups']:
            draw_label(board, data['bounds'][g[0]], (166,0,0), 'horizontal_groups')
            for compo_id in g[1:]:
                b = data['bounds'][compo_id]
                draw_label(board, b, (166,0,0), put_text=False)
                is_drawn = True
                
    if data['multitab_groups'] is not None:
        for g in data['multitab_groups']:
            draw_label(board, data['bounds'][g[0]], (166,0,166), 'multitab_groups')
            for compo_id in g[1:]:
                b = data['bounds'][compo_id]
                draw_label(board, b, (166,0,166), put_text=False)
                is_drawn = True
    
    if data['list_groups'] is not None:
        for g in data['list_groups']:
            draw_label(board, data['bounds'][g[0]], (166,100,255), 'list_groups')
            for compo_id in g[1:]:
                b = data['bounds'][compo_id]
                draw_label(board, b, (166,100,255), put_text=False)
                is_drawn = True        
                
    if data['pageindcator_groups'] is not None:
        for g in data['pageindcator_groups']:
            draw_label(board, data['bounds'][g[0]], (166,166,166), 'pageindcator_groups')
            for compo_id in g[1:]:
                b = data['bounds'][compo_id]
                draw_label(board, b, (166,166,166), put_text=False)
                is_drawn = True     
        
    if show and is_drawn:
        cv2.imshow('data', cv2.resize(board, (500, 800)))
        key = cv2.waitKey(0)
        return key
    return None


def visualize_results(image_name):
    uied_result = cv2.imread(pjoin(result_root, 'uied', image_name + '.jpg'))
    layout_result = cv2.imread(pjoin(result_root, 'layout', image_name + '-list.jpg'))
    cv2.imshow('uied', uied_result)
    cv2.imshow('layout', layout_result)
    

def visualize_ground_truth(image_name, gt_data):
    img_file = pjoin(data_root, image_name + '.jpg')
    img_data = gt_data[image_name]
    img = cv2.imread(img_file)
    
    org_json_file = pjoin(data_root, image_name + '.json')
    org_data = json.load(open(org_json_file))
    org_shape = org_data['activity']['root']['bounds']
    
    img = cv2.resize(img, (1440, 2560))
    return visualize_list(img, img_data)


def cvt_gt_data(gt_data):
    lst = sorted(gt_data['list'].items(), key=lambda x: x[0])
    lst = [l[1] for l in lst]
    return lst

In [3]:
def print_list(gt, result):
    print('*** Ground Truth ***')
    print('Number of list items:', len(gt))
    for item in gt:
        print(item)
    print('*** Result ***')
    print('Number of lists:', len(result))
    for lst in result:
        print('Number of list items:', len(lst))
        for item in lst:
            print(item)

            
def check_correctness(gt, result, is_print=False):
    correct_item_num = 0
    correct_ele_num = 0
    correct_ele_type = 0
    
    for lst in result:
        # level 1: item number in a list
        if len(gt) == len(lst):
            correct_item_num += 1

            # level 2: element number each item
            for i in range(len(gt)):
                if len(gt[i]) == len(lst[i]):
                    correct_ele_num += 1

                    # level 3: element type in each item
                    for j in range(len(gt[i])):
                        if (gt[i][j] == 'TextView' and lst[i][j] == 'Text') or (gt[i][j] != 'TextView' and lst[i][j] != 'Text'):
                            correct_ele_type += 1
            break
            
    if is_print:
        print_list(gt, result)
        print('Correct item number of the list:', correct_item_num)
        print('Correct element number of a item:', correct_ele_num, correct_ele_num / len(gt))
        print('Correct element type number of a item:', correct_ele_type / len(gt))

In [4]:
def is_correct_list(gt, result):
    correct_list_num = 0
    correct_item_num = 0
    correct_ele_type = 0
    
    for rst in result:
        # level 1: item number in a list
        if abs(len(gt) - len(rst)) <= 2:
            # level 2: element number each item
            matched_item_id = 0
            for i in range(len(gt)):
                for j in range(matched_item_id, len(rst)):
                    # compare the element number in the two list items
                    if abs(len(gt[i]) - len(rst[j])) <= 1:
                        correct_item_num += 1
                        matched_item_id = j + 1
            
            if correct_item_num == min(len(gt), len(rst)):
                return True
    return False

In [5]:
data_root = 'E:\\Mulong\\Datasets\\gui\\rico\\combined\\all'
result_root = 'E:\\Temp\\data\\rico-layout\\result'
list_data_gt = json.load(open('item_data_class.json'))
gui_gt = json.load(open('valid_group_data.json'))

In [7]:
true_positive = 0
for n, img_name in enumerate(open('select.txt', 'r').readlines()):
    img_name = img_name.replace('\n', '')
    
    list_gt = cvt_gt_data(list_data_gt[img_name])
    list_result = json.load(open(pjoin('E:\\Temp\\data\\rico-layout\\result', 'layout', img_name + '-list.json')))['list']

    is_correct = is_correct_list(list_gt, list_result)
    if is_correct:
        true_positive += 1
    print(img_name, is_correct, true_positive, '/', n)

2114 True 1 / 0
30783 True 2 / 1
41634 False 2 / 2
13614 False 2 / 3
17314 True 3 / 4
34532 False 3 / 5
30998 False 3 / 6
20553 False 3 / 7
24735 True 4 / 8
14190 False 4 / 9
26644 False 4 / 10
32619 True 5 / 11
41191 False 5 / 12
34917 False 5 / 13
15701 True 6 / 14
24537 False 6 / 15
25173 True 7 / 16
31106 False 7 / 17
9323 True 8 / 18
31099 True 9 / 19
28292 True 10 / 20
7440 True 11 / 21
35228 False 11 / 22
31728 True 12 / 23
35566 True 13 / 24
3183 True 14 / 25
30185 False 14 / 26
21088 False 14 / 27
2060 False 14 / 28
37198 True 15 / 29
12456 False 15 / 30
26903 True 16 / 31
10282 False 16 / 32
27406 False 16 / 33
31517 False 16 / 34
30609 True 17 / 35
24668 False 17 / 36
10283 True 18 / 37
30147 True 19 / 38
7131 True 20 / 39
16631 True 21 / 40
13985 True 22 / 41
8558 False 22 / 42
35573 False 22 / 43
17936 False 22 / 44
39407 True 23 / 45
13563 True 24 / 46
2505 False 24 / 47
4606 False 24 / 48
16341 True 25 / 49
24495 True 26 / 50
31107 True 27 / 51
24250 True 28 / 52
34080 F

11097 False 277 / 599
14845 True 278 / 600
19385 True 279 / 601
3513 False 279 / 602
1304 True 280 / 603
5176 True 281 / 604
35876 True 282 / 605
34598 True 283 / 606
692 True 284 / 607
28614 False 284 / 608
33552 False 284 / 609
2394 False 284 / 610
1689 True 285 / 611
23225 False 285 / 612
40635 False 285 / 613
6045 True 286 / 614
19032 True 287 / 615
4120 True 288 / 616
34188 False 288 / 617
7373 False 288 / 618
35282 False 288 / 619
7415 False 288 / 620
527 True 289 / 621
8083 False 289 / 622
36748 True 290 / 623
2747 True 291 / 624
9148 True 292 / 625
24199 False 292 / 626
11122 True 293 / 627
23609 False 293 / 628
20472 False 293 / 629
29330 True 294 / 630
21745 False 294 / 631
28761 False 294 / 632
6285 True 295 / 633
22270 True 296 / 634
15351 True 297 / 635
3470 True 298 / 636
18891 True 299 / 637
3458 True 300 / 638
19966 False 300 / 639
39320 False 300 / 640
19782 True 301 / 641
26214 False 301 / 642
20842 False 301 / 643
22648 False 301 / 644
38162 False 301 / 645
17817 Tru

15479 False 467 / 999
11637 True 468 / 1000
24840 False 468 / 1001
13752 False 468 / 1002
5073 True 469 / 1003
25009 False 469 / 1004
12841 True 470 / 1005
13578 False 470 / 1006
9477 False 470 / 1007
18377 False 470 / 1008
7300 False 470 / 1009
37579 True 471 / 1010
20777 True 472 / 1011
23530 True 473 / 1012
4966 False 473 / 1013
29784 True 474 / 1014
11569 True 475 / 1015
3006 False 475 / 1016
36840 True 476 / 1017
3012 True 477 / 1018
2683 False 477 / 1019
24713 False 477 / 1020
15280 True 478 / 1021
27540 True 479 / 1022
10307 False 479 / 1023
9701 True 480 / 1024
79 False 480 / 1025
6418 True 481 / 1026
3560 False 481 / 1027
37792 True 482 / 1028
23081 False 482 / 1029
19309 False 482 / 1030
8390 False 482 / 1031
35152 True 483 / 1032
34258 False 483 / 1033
22374 False 483 / 1034
32615 True 484 / 1035
24711 True 485 / 1036
41412 False 485 / 1037
21480 True 486 / 1038
13340 True 487 / 1039
17898 True 488 / 1040
16210 False 488 / 1041
29792 True 489 / 1042
4970 False 489 / 1043
214

In [6]:
start_point = None
start = False
for img_name in open('select.txt', 'r').readlines()[:200]:
    img_name = img_name.replace('\n', '')
    if start_point is None or img_name == start_point: start = True
    if not start: continue
    print('\n', img_name)
    
    list_gt = cvt_gt_data(list_data_gt[img_name])
    list_result = json.load(open(pjoin('E:\\Temp\\data\\rico-layout\\result', 'layout', img_name + '-list.json')))['list']

    check_correctness(list_gt, list_result, is_print=True)
    print(is_correct_list(list_gt, list_result))
    
    visualize_results(img_name)
    key = visualize_ground_truth(img_name, gui_gt)
    if key is not None and key != -1 and chr(key) == 'q':
        break
cv2.destroyAllWindows()


 2114
*** Ground Truth ***
Number of list items: 10
['TextView', 'TextView']
['TextView', 'TextView']
['TextView', 'TextView', 'TextView']
['TextView', 'TextView']
['TextView']
['TextView', 'TextView']
['TextView']
['TextView', 'TextView']
['TextView', 'TextView']
['TextView', 'TextView']
*** Result ***
Number of lists: 1
Number of list items: 10
['Compo', 'Text']
['Compo', 'Text']
['Compo', 'Text']
['Compo', 'Text']
['Text']
['Compo', 'Text']
['Text']
['Compo', 'Text']
['Compo', 'Text']
['Compo', 'Text']
Correct item number of the list: 1
Correct element number of a item: 9 0.9
Correct element type number of a item: 0.9
True

 30783
*** Ground Truth ***
Number of list items: 8
['TextView']
['TextView']
['TextView']
['TextView']
['TextView']
['TextView']
['TextView']
['TextView', 'TextView']
*** Result ***
Number of lists: 1
Number of list items: 8
['Text']
['Text']
['Text']
['Text']
['Text']
['Text']
['Text']
['Text']
Correct item number of the list: 1
Correct element number of a ite

In [7]:
img_name = '2114'

list_gt = cvt_list_gt(list_data_gt[img_name])
list_result = json.load(open(pjoin('E:\\Temp\\data\\rico-layout\\result', 'layout', img_name + '-list.json')))['list']

evaluate(list_gt, list_result, is_print=True)
print(is_correct_list(list_gt, list_result))

visualize_results(img_name)
visualize_ground_truth(img_name, gui_gt)
cv2.destroyAllWindows()

*** Ground Truth ***
Number of list items: 10
['TextView', 'TextView']
['TextView', 'TextView']
['TextView', 'TextView', 'TextView']
['TextView', 'TextView']
['TextView']
['TextView', 'TextView']
['TextView']
['TextView', 'TextView']
['TextView', 'TextView']
['TextView', 'TextView']
*** Result ***
Number of lists: 1
Number of list items: 10
['Compo', 'Text']
['Compo', 'Text']
['Compo', 'Text']
['Compo', 'Text']
['Text']
['Compo', 'Text']
['Text']
['Compo', 'Text']
['Compo', 'Text']
['Compo', 'Text']
Correct item number of the list: 1
Correct element number of a item: 9 0.9
Correct element type number of a item: 0.9
10
True
